In [1]:
import requests
from datetime import datetime
import pandas as pd
import re
import time
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
import itertools
from pathlib import Path
import seaborn as sns
from matplotlib import pyplot as plt
import yaml
import ast
import glob
import numpy as np
import psycopg2
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int32, psycopg2._psycopg.AsIs)
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)

In [93]:
# Время ожидания между запросами, в сек.
wait = 0.3

# Функции подключения к PostgreSQL.
param_dic = {
    'host'      : '192.168.200.12',
    'database'  : 'hh_pet',
    'user'      : 'hh_user',
    'password'  : 'DWuh1289h12ej2h32178dh2-38hD'
}

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    print("Connection successful")
    return conn

def execute_mogrify_insert(conn, df, table):
    """
    Using cursor.mogrify() to build the bulk insert query
    then cursor.execute() to execute the query
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    cursor = conn.cursor()
    
    col = ''
    for i in range(0, len(df.columns)):
        col += '%s,'
    values = [cursor.mogrify("(" + col[:-1] + ")", tup).decode('utf8') for tup in tuples]
    query  = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)
    
    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_mogrify() done")
    cursor.close()

In [ ]:
# Загрузим все специализации в базу.
url = 'https://api.hh.ru/specializations'
par = {'User-Agent': 'Pet project (some-mail@yandex.ru)'}
response = requests.get(url, params=par).json()

In [24]:
dataDict = {}
for j in range(0,len(response)):
    dataDict[j] = dict(response[j])

#Создаем DataFrame из dict (словаря данных или массива данных)
keysDict = dataDict[0].keys()
df_spec = pd.DataFrame.from_dict(dataDict, orient='index',columns=keysDict)
df_spec.head()

,id,name,specializations
0,1,"Информационные технологии, интернет, телеком","[{'id': '1.395', 'name': 'Банковское ПО', 'lab..."
1,20,"Строительство, недвижимость","[{'id': '20.396', 'name': 'Эксплуатация', 'lab..."
2,17,Продажи,"[{'id': '17.397', 'name': 'Электротехническое ..."
3,13,"Медицина, фармацевтика","[{'id': '13.398', 'name': 'Клинические исследо..."
4,5,"Банки, инвестиции, лизинг","[{'id': '5.399', 'name': 'Ипотека, Ипотечное к..."


In [25]:
df_spec2 = pd.DataFrame()
dataDict = {}
for i in range(0,len(df_spec)):
    k = 0
    for j in range(len(dataDict),len(dataDict)+len(df_spec.loc[i,'specializations'])):
        dataDict[j] = dict(df_spec.loc[i,'specializations'][k])
        k += 1
    
keysDict = dataDict[0].keys()
tmp = pd.DataFrame.from_dict(dataDict, orient='index',columns=keysDict)
tmp['parent_id'] = tmp['id'].apply(lambda x: x.split('.')[0])
tmp.merge(df_spec, how='left', left_on='parent_id', right_on='id')
if len(df_spec2) == 0:
    df_spec2 = tmp
else:
    pd.concat([df_spec2,tmp])


In [26]:
df_spec = df_spec2.merge(df_spec, how='left', left_on='parent_id', right_on='id')
df_spec.head()

,id_x,name_x,laboring,parent_id,id_y,name_y,specializations
0,1.395,Банковское ПО,False,1,1,"Информационные технологии, интернет, телеком","[{'id': '1.395', 'name': 'Банковское ПО', 'lab..."
1,1.400,Оптимизация сайта (SEO),False,1,1,"Информационные технологии, интернет, телеком","[{'id': '1.395', 'name': 'Банковское ПО', 'lab..."
2,1.420,Администратор баз данных,False,1,1,"Информационные технологии, интернет, телеком","[{'id': '1.395', 'name': 'Банковское ПО', 'lab..."
3,1.474,Стартапы,False,1,1,"Информационные технологии, интернет, телеком","[{'id': '1.395', 'name': 'Банковское ПО', 'lab..."
4,1.475,Игровое ПО,False,1,1,"Информационные технологии, интернет, телеком","[{'id': '1.395', 'name': 'Банковское ПО', 'lab..."


In [31]:
df_spec.drop(columns=['laboring', 'id_y', 'specializations'], inplace=True)
df_spec.rename(columns={'id_x':'spec_id', 'name_x':'name', 'name_y':'parent_name'}, inplace=True)

In [74]:
# Записали данные в БД.
conn = connect(param_dic)
execute_mogrify_insert(conn, df_spec, 'specializations')
conn.close()

Connecting to the PostgreSQL database...
Connection successful
execute_mogrify() done


In [345]:
# Загрузим все города в базу.
url = 'https://api.hh.ru/areas/'
par = {'User-Agent': 'Pet project (some-mail@yandex.ru)'}
response = requests.get(url, params=par).json()

In [346]:
dataDict = {}
for j in range(0,len(response)):
    dataDict[j] = dict(response[j])

#Создаем DataFrame из dict (словаря данных или массива данных)
keysDict = dataDict[0].keys()
df_area = pd.DataFrame.from_dict(dataDict, orient='index',columns=keysDict)
df_area = pd.DataFrame(df_area.loc[0,'areas'])
df_area.head()

,id,parent_id,name,areas
0,1620,113,Республика Марий Эл,"[{'id': '4228', 'parent_id': '1620', 'name': '..."
1,1624,113,Республика Татарстан,"[{'id': '1625', 'parent_id': '1624', 'name': '..."
2,1646,113,Удмуртская Республика,"[{'id': '4192', 'parent_id': '1646', 'name': '..."
3,1652,113,Чувашская Республика,"[{'id': '1653', 'parent_id': '1652', 'name': '..."
4,1192,113,Забайкальский край,"[{'id': '216', 'parent_id': '1192', 'name': 'А..."


In [347]:
df_area2 = pd.DataFrame()
dataDict = {}
for i in range(0,len(df_area)):
    k = 0
    cnt_from = len(dataDict)
    cnt_to = len(dataDict)+len(df_area.loc[i,'areas'])
    for j in range(cnt_from,cnt_to):
        dataDict[j] = dict(df_area.loc[i,'areas'][k])
        k += 1
    
keysDict = dataDict[0].keys()
tmp = pd.DataFrame.from_dict(dataDict, orient='index',columns=keysDict)
#tmp['parent_id'] = tmp['id'].apply(lambda x: x.split('.')[0])
tmp.merge(df_area, how='left', left_on='parent_id', right_on='id')
if len(df_area2) == 0:
    df_area2 = tmp
else:
    pd.concat([df_area2,tmp])

In [348]:
df_area2.drop(columns=['areas'], inplace=True)

In [349]:
df_area2.head()

,id,parent_id,name
0,4228,1620,Виловатово
1,1621,1620,Волжск
2,1622,1620,Звенигово
3,4229,1620,Знаменский
4,61,1620,Йошкар-Ола


In [350]:
df_area['length'] = df_area['areas'].apply(len)
df_area.head()

,id,parent_id,name,areas,length
0,1620,113,Республика Марий Эл,"[{'id': '4228', 'parent_id': '1620', 'name': '...",21
1,1624,113,Республика Татарстан,"[{'id': '1625', 'parent_id': '1624', 'name': '...",62
2,1646,113,Удмуртская Республика,"[{'id': '4192', 'parent_id': '1646', 'name': '...",33
3,1652,113,Чувашская Республика,"[{'id': '1653', 'parent_id': '1652', 'name': '...",30
4,1192,113,Забайкальский край,"[{'id': '216', 'parent_id': '1192', 'name': 'А...",42


In [351]:
df_area[df_area['length']==0]

,id,parent_id,name,areas,length
20,1,113,Москва,[],0
39,2,113,Санкт-Петербург,[],0


Два города, у которых Родительский регион "Россия" - это надо исправить. У Москвы и Питера свои регионы.

In [352]:
display(df_area[df_area['name']=='Московская область'])
display(df_area[df_area['name']=='Ленинградская область'])

,id,parent_id,name,areas,length
21,2019,113,Московская область,"[{'id': '6339', 'parent_id': '2019', 'name': '...",207


,id,parent_id,name,areas,length
32,145,113,Ленинградская область,"[{'id': '5408', 'parent_id': '145', 'name': 'А...",134


Добавим данные по этим городам в самом конце.

In [353]:
df_area.drop(columns=['parent_id','areas','length'], inplace=True)
df_area.rename(columns={'id':'region_id','name':'region'}, inplace=True)
df_area2.rename(columns={'parent_id':'region_id'}, inplace=True)

In [354]:
df_area = df_area2.merge(df_area, how='left', on='region_id')
df_area.head()

,id,region_id,name,region
0,4228,1620,Виловатово,Республика Марий Эл
1,1621,1620,Волжск,Республика Марий Эл
2,1622,1620,Звенигово,Республика Марий Эл
3,4229,1620,Знаменский,Республика Марий Эл
4,61,1620,Йошкар-Ола,Республика Марий Эл


In [355]:
df_area.loc[:,'id'] = df_area['id'].apply(int)
df_area.loc[:,'region_id'] = df_area['region_id'].apply(int)

In [356]:
# Загружу подготовленные данные по городам с информацией по количеству населения и по средним ЗП по региону (данные Росстат).
df_cities = pd.read_excel('cities_data.xlsx')
df_cities

,region,city,population,point_lon,point_lat,salary_avg_month
0,Республика Адыгея,Абадзехская,3693,44.390830,40.22333,26678
1,Республика Адыгея,Адамий,1406,45.068890,39.49750,26678
2,Республика Адыгея,Адыгейск,12721,44.884525,39.19202,26678
3,Республика Адыгея,Ассоколай,1442,44.845000,39.46806,26678
4,Республика Адыгея,Афипсип,2225,44.993330,38.77944,26678
...,...,...,...,...,...,...
9582,Севастополь,Солнечный,1718,44.786110,33.61667,30628
9583,Севастополь,Терновка,1843,44.581940,33.74444,30628
9584,Севастополь,Фронтовое,1006,44.668060,33.73611,30628
9585,Севастополь,Штурмовое,1535,44.568060,33.62222,30628


In [357]:
# Отличия в названии регионов
df_area[~df_area['region'].isin(df_cities['region'].unique())]['region'].unique()

array(['Удмуртская Республика', 'Чувашская Республика',
       'Кабардино-Балкарская республика',
       'Карачаево-Черкесская Республика',
       'Республика Северная Осетия-Алания', 'Чеченская республика'],
      dtype=object)

In [358]:
df_cities['region'].unique()

array(['Республика Адыгея', 'Республика Башкортостан',
       'Республика Бурятия', 'Республика Алтай', 'Республика Дагестан',
       'Республика Ингушетия', 'Республика Кабардино-Балкария',
       'Республика Калмыкия', 'Республика Карачаево-Черкесия',
       'Республика Карелия', 'Республика Коми', 'Республика Марий Эл',
       'Республика Мордовия', 'Республика Саха (Якутия)',
       'Республика Северная Осетия — Алания', 'Республика Татарстан',
       'Республика Тыва', 'Республика Удмуртия', 'Республика Хакасия',
       'Республика Чечня', 'Республика Чувашия', 'Алтайский край',
       'Краснодарский край', 'Красноярский край', 'Приморский край',
       'Ставропольский край', 'Хабаровский край', 'Амурская область',
       'Архангельская область', 'Астраханская область',
       'Белгородская область', 'Брянская область', 'Владимирская область',
       'Волгоградская область', 'Вологодская область',
       'Воронежская область', 'Ивановская область', 'Иркутская область',
       'Кал

In [359]:
# Изменим названия регионов в файле на те, которые в hh.
df_cities.loc[df_cities['region']=='Республика Удмуртия','region'] = 'Удмуртская Республика'
df_cities.loc[df_cities['region']=='Республика Чувашия','region'] = 'Чувашская Республика'
df_cities.loc[df_cities['region']=='Республика Кабардино-Балкария','region'] = 'Кабардино-Балкарская республика'
df_cities.loc[df_cities['region']=='Республика Карачаево-Черкесия','region'] = 'Карачаево-Черкесская Республика'
df_cities.loc[df_cities['region']=='Республика Северная Осетия — Алания','region'] = 'Республика Северная Осетия-Алания'
df_cities.loc[df_cities['region']=='Республика Чечня','region'] = 'Чеченская республика'

# Еще раз проверим отличия
df_area[~df_area['region'].isin(df_cities['region'].unique())]['region'].unique()

array([], dtype=object)

In [360]:
df_area.rename(columns={'name':'city'}, inplace=True)

In [361]:
df_area.head()

,id,region_id,city,region
0,4228,1620,Виловатово,Республика Марий Эл
1,1621,1620,Волжск,Республика Марий Эл
2,1622,1620,Звенигово,Республика Марий Эл
3,4229,1620,Знаменский,Республика Марий Эл
4,61,1620,Йошкар-Ола,Республика Марий Эл


In [362]:
df_cities.head()

,region,city,population,point_lon,point_lat,salary_avg_month
0,Республика Адыгея,Абадзехская,3693,44.390830,40.22333,26678
1,Республика Адыгея,Адамий,1406,45.068890,39.49750,26678
2,Республика Адыгея,Адыгейск,12721,44.884525,39.19202,26678
3,Республика Адыгея,Ассоколай,1442,44.845000,39.46806,26678
4,Республика Адыгея,Афипсип,2225,44.993330,38.77944,26678


In [363]:
df_area[df_area['city'].str.contains('\)')]

,id,region_id,city,region
8,4232,1620,Красногорский (Республика Марий Эл),Республика Марий Эл
17,4240,1620,Приволжский (Республика Марий Эл),Республика Марий Эл
19,4242,1620,Советский (Республика Марий Эл),Республика Марий Эл
46,1635,1624,Зеленодольск (Республика Татарстан),Республика Татарстан
51,4178,1624,Карабаш (Республика Татарстан),Республика Татарстан
...,...,...,...,...
3855,5994,1530,Овощной (Ростовская область),Ростовская область
3856,4541,1530,Ольгинская (Ростовская область),Ростовская область
3861,4545,1530,Покровское (Ростовская область),Ростовская область
3876,5301,1530,Советская (Ростовская область),Ростовская область


Много записей с дублированными регионами в названии городов. Понятно для чего это делалось, но мне это мешает. Уберём их.

In [364]:
df_area.loc[df_area['city'].str.contains('\)'),'city'] = df_area.loc[df_area['city'].str.contains('\)'), 'city'].str.replace('\s\(.+\)', '', regex=True)
df_area.loc[df_area['city'].str.contains('\)'),'city'] = df_area.loc[df_area['city'].str.contains('\)'), 'city'].str.replace('\(.+\)', '', regex=True)

In [365]:
print('Кол-во записей в списке городов с hh:', len(df_area))
df_area_result = df_area.merge(df_cities, how='left',on=['city','region'])
print('После объединения:', len(df_area_result))
df_area_result.info()

Кол-во записей в списке городов с hh: 3891
После объединения: 3982
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3982 entries, 0 to 3981
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                3982 non-null   int64  
 1   region_id         3982 non-null   int64  
 2   city              3982 non-null   object 
 3   region            3982 non-null   object 
 4   population        3600 non-null   float64
 5   point_lon         3600 non-null   float64
 6   point_lat         3600 non-null   float64
 7   salary_avg_month  3600 non-null   float64
dtypes: float64(4), int64(2), object(2)
memory usage: 280.0+ KB


In [366]:
# Кол-во городов увеличилось и по 5 городам не нашлись данные. Посмотрим на это более подробно.
df_area_result[df_area_result.duplicated(['id','region_id'], keep=False)]

,id,region_id,city,region,population,point_lon,point_lat,salary_avg_month
42,4176,1624,Высокая Гора,Республика Татарстан,2968.0,55.933100,49.308100,33520.0
43,4176,1624,Высокая Гора,Республика Татарстан,9589.0,55.912780,49.301670,33520.0
205,4860,1124,Железнодорожный,Иркутская область,1361.0,52.672500,103.753610,41495.0
206,4860,1124,Железнодорожный,Иркутская область,6463.0,57.908330,102.766670,41495.0
261,1149,1146,Боготол,Красноярский край,19576.0,56.200000,89.516670,44922.0
...,...,...,...,...,...,...,...,...
3826,4587,1511,Октябрьский,Волгоградская область,5522.0,47.966670,43.643610,29708.0
3873,4492,1530,Александровка,Ростовская область,4744.0,46.783060,38.998610,30053.0
3874,4492,1530,Александровка,Ростовская область,1349.0,47.327877,40.001503,30053.0
3931,1543,1530,Миллерово,Ростовская область,34464.0,48.916670,40.400000,30053.0


In [367]:
df_cities[df_cities['city']=='Боготол']

,region,city,population,point_lon,point_lat,salary_avg_month
2935,Красноярский край,Боготол,19576,56.20000,89.51667,44922
2936,Красноярский край,Боготол,1992,56.17194,89.56889,44922


In [368]:
df_cities[df_cities['city']=='Раздольное']

,region,city,population,point_lon,point_lat,salary_avg_month
1561,Республика Северная Осетия-Алания,Раздольное,1004,43.698330,44.535280,25618
2772,Краснодарский край,Раздольное,2243,43.591110,39.767780,32096
2773,Краснодарский край,Раздольное,1081,46.711892,39.665525,32096
3187,Приморский край,Раздольное,8194,43.533330,131.897220,41728
3478,Ставропольский край,Раздольное,2221,45.331110,41.296940,28536
3698,Амурская область,Раздольное,1750,50.018330,127.826611,43367
4396,Волгоградская область,Раздольное,1451,49.915000,46.061110,29708
6641,Новосибирская область,Раздольное,4861,55.067170,83.240660,34499
9498,Республика Крым,Раздольное,7231,45.769440,33.486110,28916
9499,Республика Крым,Раздольное,1008,45.361110,34.854170,28916


Населённые пункты с одним названием есть в одном регионе. Т.к. анализ по средним зарплатам проводится в разрезе регионов, то убёрем все дубликаты, оставив населённые пункты с максимальным населением.

In [369]:
df_area_result = df_area_result.sort_values('population', ascending=False).reset_index(drop=True)
df_area_result

,id,region_id,city,region,population,point_lon,point_lat,salary_avg_month
0,4,1202,Новосибирск,Новосибирская область,1625631.0,55.01667,82.91667,34499.0
1,3,1261,Екатеринбург,Свердловская область,1493749.0,56.83333,60.58333,36553.0
2,88,1624,Казань,Республика Татарстан,1257391.0,55.79083,49.11444,33520.0
3,66,1679,Нижний Новгород,Нижегородская область,1252236.0,56.32694,44.00750,31859.0
4,104,1384,Челябинск,Челябинская область,1196680.0,55.16222,61.40306,32953.0
...,...,...,...,...,...,...,...,...
3977,4532,1530,Мечетинская,Ростовская область,NaN,NaN,NaN,NaN
3978,5996,1530,Песчаный,Ростовская область,NaN,NaN,NaN,NaN
3979,5995,1530,Новониколаевка,Ростовская область,NaN,NaN,NaN,NaN
3980,4536,1530,Новониколаевский,Ростовская область,NaN,NaN,NaN,NaN


In [370]:
# Уберём дубликаты и посмотрим количество городов. Должно сходиться с изначальными данными hh.
# Почему не по city и region: в исходных данных с hh.ru присутствует изначально 3 дубликата
# по городу и региону с разными id.
df_area_result = df_area_result.drop_duplicates(['id','region_id'], keep='first')
print('В загрузке с hh городов:', len(df_area))
print('После наших дополнений:', len(df_area_result))
df_area_result.head()

В загрузке с hh городов: 3891
После наших дополнений: 3891


,id,region_id,city,region,population,point_lon,point_lat,salary_avg_month
0,4,1202,Новосибирск,Новосибирская область,1625631.0,55.01667,82.91667,34499.0
1,3,1261,Екатеринбург,Свердловская область,1493749.0,56.83333,60.58333,36553.0
2,88,1624,Казань,Республика Татарстан,1257391.0,55.79083,49.11444,33520.0
3,66,1679,Нижний Новгород,Нижегородская область,1252236.0,56.32694,44.00750,31859.0
4,104,1384,Челябинск,Челябинская область,1196680.0,55.16222,61.40306,32953.0


In [376]:
# Москву и питер добавим вручную.
df_area_result.loc[3982] = [1,2019,'Москва','Московская область',12678079,55.75583,37.61778,84472]

In [372]:
df_area_result.loc[3983] = [2,145,'Санкт-Петербург','Ленинградская область',4567086,59.95,30.31667,57265]

In [373]:
df_area_result[df_area_result['population'].isnull()]

,id,region_id,city,region,population,point_lon,point_lat,salary_avg_month
3600,4230,1620,Кельмаксола,Республика Марий Эл,NaN,NaN,NaN,NaN
3601,4167,1624,Айша,Республика Татарстан,NaN,NaN,NaN,NaN
3602,2734,1624,Иннополис,Республика Татарстан,NaN,NaN,NaN,NaN
3603,4188,1624,Убей,Республика Татарстан,NaN,NaN,NaN,NaN
3604,4190,1624,Чувашское Дрожжаное,Республика Татарстан,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
3977,4532,1530,Мечетинская,Ростовская область,NaN,NaN,NaN,NaN
3978,5996,1530,Песчаный,Ростовская область,NaN,NaN,NaN,NaN
3979,5995,1530,Новониколаевка,Ростовская область,NaN,NaN,NaN,NaN
3980,4536,1530,Новониколаевский,Ростовская область,NaN,NaN,NaN,NaN


В подготовленных мною данных по населенности и средней зп данных нет. Поэтому просто дополним данные города информацией по средней зп по региону. В последствии я еще поработал с конечными данными, поискав дополнительную информацию по городам в базе Росстат. Для быстроты и больше наглядности я этим занимался в GoogleSpreadsheets: дополнял информацию по численности населения и исправлял расхождения, ориентируясь на данные Росстат.

Интересно, что приличное количество городов в тех данных что я нашёл в интернете просто отсутствуют в официальной отчетности Росстата по численности населения за 2020 год.

In [381]:
salary = df_area_result[df_area_result['population'].notnull()].groupby('region')['salary_avg_month'].mean().reset_index()
salary.head()

,region,salary_avg_month
0,Алтайский край,25050.0
1,Амурская область,43367.0
2,Архангельская область,46449.0
3,Астраханская область,32604.0
4,Белгородская область,31183.0


In [ ]:
def make_default_salary(data):
    if data[1]>0:
        return data[1]
    else:
        return salary[salary['region']==data[0]].iloc[0,1]

df_area_result.loc[:,'salary_avg_month'] = df_area_result[['region','salary_avg_month']].apply(make_default_salary, axis=1)

In [391]:
df_area_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3893 entries, 0 to 3983
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                3893 non-null   int64  
 1   region_id         3893 non-null   int64  
 2   city              3893 non-null   object 
 3   region            3893 non-null   object 
 4   population        3511 non-null   float64
 5   point_lon         3511 non-null   float64
 6   point_lat         3511 non-null   float64
 7   salary_avg_month  3893 non-null   float64
dtypes: float64(4), int64(2), object(2)
memory usage: 402.8+ KB


In [394]:
df_area_result = df_area_result.rename(columns={'id':'city_id'}).sort_values('city_id')

In [395]:
# Выгрузил данные по городом в файл.
df_area_result.to_excel('cities.xlsx', sheet_name='Лист2', index=False)